In [1]:
import pandas as pd
import re
import nltk
from pymystem3 import Mystem
from string import punctuation

In [ ]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')
punctuation = set(punctuation + ' ')
regex = re.compile(r'[А-ЯЁа-яёA-z-]+')
m = Mystem()
columns = pd.read_csv('..\\Датасеты\\news_test.csv').columns
tags = columns[2:]

df = pd.DataFrame(columns=columns)

def words_only(text, regex=regex):
    '''
    Очистка текста от наборов символов, не являющихся словами,
    и приведение к нижнему регистру
    '''
    try:
        return ' '.join(regex.findall(text)).lower()
    except:
        return ''

def preprocess_text(text):
    '''
    Лемматизация текста и его очистка от стоп-слов
    '''
    text = words_only(text)
    tokens = m.lemmatize(text)
    
    # Удаление токенов, являющихся стоп-словами, либо набором знаков пунктуации
    tokens = [token for token in tokens if token not in stop_words and not set(token.strip()) <= punctuation]
    
    text = ' '.join(tokens)
    return text

def make_example(tags_list, text):
    '''
    Добавления объекта в датасет с примерами. Заполнение столбцов
    с текстом, лемматизированным текстом, и жанрами.
    '''
    row_no = len(df)
    df.loc[row_no, 'text'] = text
    df.loc[row_no, 'text_lemma'] = preprocess_text(text)
    
    for tag in tags:
        if tag not in tags_list:
            df.loc[row_no, tag] = 0
        else:
            df.loc[row_no, tag] = 1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Paul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# данные для подключения 

username = 'postgres'  # DB username
password = '111'  # DB password
host = 'localhost'  # Public IP address for your instance
port = '5432'  # Port
database = 'postgres'  # Name of database
dbschema = 'news_db'  # Searches left-to-right

In [11]:
import psycopg2 as ps

# инструмент подключения через psycopg2
connection_pg = ps.connect(database=database,user=username, password=password, host=host, port=port)
cursor_pg = connection_pg.cursor()
cursor_pg.execute('set search_path to news_db')
cursor_pg.execute('select tags from news_table group by tags')

data_pg = cursor_pg.fetchall()
try:
    pd.DataFrame(columns = ['id','url','title','text','topic','tags','date'], data = data_pg)
except:
    print(data_pg[:5])
del(data_pg)

[('69-я параллель',), ('Coцсети',), ('Авто',), ('Автобизнес',), ('Аналитика рынка',)]


In [14]:
import os
import sqlalchemy
from sqlalchemy import event
import pandas as pd
import psycopg2 as ps
import warnings
warnings.simplefilter("ignore")
# инструмент подключения через sqlalchemy

db_url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, host, port, database)
engine = sqlalchemy.create_engine(db_url)


# установка search_path для всех транзакций в функционале sqlalchemy

@event.listens_for(engine, "connect", insert=True)
def set_search_path(dbapi_connection, connection_record):
    existing_autocommit = dbapi_connection.autocommit
    dbapi_connection.autocommit = True
    cursor = dbapi_connection.cursor()
    cursor.execute("SET SESSION search_path='%s'" % dbschema)
    cursor.close()
    dbapi_connection.autocommit = existing_autocommit

In [ ]:
try:
    os.mkdir('..\\Датасеты\\examples')
except:
    print('Already exists')

In [ ]:
# проход по полученному списку тегов и выбор первых 2 строк по каждому тегу при сортировке по id по убыванию
sum_tags = 0
for tag in columns[2:]:
    query = f'''
    select * from news_table nt 
    where tags = '{tag}'
    order by id desc
    limit 3
    '''
    # сохранение данных по каждому тегу в датафрейм
    df_tag_example = pd.read_sql_query(query, con = engine)
    len_df = len(df_tag_example)
    sum_tags += len_df
    # сохранение полученного датафрейма по каждому тегу в csv файл с указанием количества строк
    df_tag_example.to_csv(f'..\\Датасеты\\examples\\{tag}_{len_df}_ex.csv', index = False)

In [ ]:
for file in os.listdir('..\\Датасеты\\examples\\'):
    print(file)
    df_file = pd.read_csv('..\\Датасеты\\examples\\'+file)
    df_file.drop(columns = ['url','title', 'topic', 'date','year'], inplace = True)
    for row in range(len(df_file)):
        tags_list = [df_file['tags'][row]]
        text = f'''{df_file['text'][row]}'''
        make_example(tags_list, text)
    

Авто_3_ex.csv
Автобизнес_3_ex.csv
Белоруссия_3_ex.csv
Бизнес_3_ex.csv
Бокс и ММА_3_ex.csv
Бывший СССР_3_ex.csv
Вещи_3_ex.csv
Вирусные ролики_3_ex.csv
Вкусы_3_ex.csv
Внешний вид_3_ex.csv
Гаджеты_3_ex.csv
Город_3_ex.csv
Госэкономика_3_ex.csv
Дача_3_ex.csv
Движение_3_ex.csv
Деловой климат_3_ex.csv
Деньги_3_ex.csv
Дом_3_ex.csv
Достижения_3_ex.csv
Еда_3_ex.csv
Жизнь_3_ex.csv
Закавказье_3_ex.csv
Звери_3_ex.csv
Зимние виды_3_ex.csv
Игры_3_ex.csv
Из жизни_3_ex.csv
Инструменты_3_ex.csv
Интернет и СМИ_3_ex.csv
Интернет_3_ex.csv
Искусство_3_ex.csv
История_3_ex.csv
Квартира_3_ex.csv
Киберпреступность_3_ex.csv
Кино_3_ex.csv
Книги_3_ex.csv
Конфликты_3_ex.csv
Космос_3_ex.csv
Криминал_3_ex.csv
Крым_3_ex.csv
Культура_3_ex.csv
Летние виды_3_ex.csv
Люди_3_ex.csv
Мемы_3_ex.csv
Мир_3_ex.csv
Мировой бизнес_3_ex.csv
Мнения_3_ex.csv
Молдавия_3_ex.csv
Москва_3_ex.csv
Музыка_3_ex.csv
Наука и техника_3_ex.csv
Наука_3_ex.csv
Общество_3_ex.csv
Оружие_3_ex.csv
Офис_3_ex.csv
Первая мировая_3_ex.csv
Политика_3_ex.csv

In [ ]:
tags_list = []
text = """

"""

make_example(tags_list, text)


In [35]:
df

,text_lemma,Coцсети,Авто,Автобизнес,Белоруссия,Бизнес,Бокс и ММА,Бывший СССР,Вещи,Вирусные ролики,...,Украина,Финансы компаний,Футбол,Хоккей,Ценности,Часы,Экология,Экономика,Явления,text
0,пилот команда русский время артем маркелов выс...,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Пилот команды «Русское время» Артем Маркелов, ..."
1,архиепископ георг генсвайн рассказывать встреч...,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Архиепископ Георг Генсвайн рассказал о встрече...
2,редакция британский телеканал sky sports свой ...,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Редакция британского телеканала Sky Sports на ...
3,первый украинский суперкар himera q замечать а...,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Первый украинский суперкар Himera Q заметили н...
4,генеральный директор ferrari nv серджио маркио...,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Генеральный директор Ferrari NV Серджио Маркио...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,председатель совет директор альфа группа михаи...,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Председатель совета директоров «Альфа Групп» М...
260,пенсионный фонд россия пфр рассказывать размер...,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,В Пенсионном фонде России (ПФР) рассказали о р...
261,южнокорейский компания производство косметика ...,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Южнокорейская компания по производству космети...
262,книга список абонент автоматический телефонный...,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Книгу со списком абонентов автоматической теле...


In [ ]:
df.drop('text', axis=1).to_csv('..\\Датасеты\\example.csv', index=False)